In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.vector_ar.vecm import VECM, select_order, select_coint_rank
from statsmodels.tsa.stattools import adfuller
import yfinance as yf

In [ ]:
stocks = ["BBCA.JK", "TLKM.JK"]  # Kode saham di Bursa Efek Indonesia (BEI)
data = yf.download(stocks, start="2020-01-01", end="2024-01-01")

# Gunakan hanya harga penutupan (Close Price)
df = data["Close"]
df = df.dropna()  # Hapus data yang hilang

# Tampilkan 5 baris pertama
print(df.head())

In [ ]:
def adf_test(series):
    result = adfuller(series)
    print(f"ADF Statistic: {result[0]}")
    print(f"p-value: {result[1]}")
    print("Stationary" if result[1] < 0.05 else "Non-Stationary")

# Uji stasioneritas untuk setiap variabel
print("Stock_A:")
adf_test(df["Stock_A"])
print("\nStock_B:")
adf_test(df["Stock_B"])

In [ ]:
Jika data tidak stasioner, kita ambil first difference:
df_diff = df.diff().dropna()

In [ ]:
# Menentukan lag optimal
lag_selection = select_order(df, maxlags=10, deterministic="ci")
print(lag_selection.aic)

# Menentukan rank kointegrasi
coint_rank = select_coint_rank(df, det_order=0, k_ar_diff=lag_selection.aic)
print(coint_rank.rank)

In [ ]:
vecm = VECM(df, k_ar_diff=lag_selection.aic, coint_rank=coint_rank.rank, deterministic="ci")
vecm_res = vecm.fit()

In [ ]:
Sekarang, kita akan melakukan prediksi untuk 10 periode ke depan:

n_forecast = 10
forecast = vecm_res.forecast(df.values[-vecm.k_ar:], steps=n_forecast)

# Membuat dataframe hasil prediksi
forecast_index = pd.date_range(start=df.index[-1], periods=n_forecast+1, freq="D")[1:]
forecast_df = pd.DataFrame(forecast, index=forecast_index, columns=df.columns)

In [ ]:
# Visualisasi hasil prediksi
plt.figure(figsize=(10, 5))
plt.plot(df.index, df["Stock_A"], label="Actual Stock_A")
plt.plot(forecast_df.index, forecast_df["Stock_A"], label="Forecast Stock_A", linestyle="dashed")
plt.legend()
plt.show()